In [2]:
import pandas as pd
import numpy as np

In [3]:
file_path = '/Users/diana/Desktop/isear-train.xlsx'
custom_headers = ['Emotions', 'Text']
df = pd.read_excel(file_path, skiprows=1, header=None, names=custom_headers)

conda install openpyxl

In [4]:
df.head ()

,Emotions,Text
0,sadness,Losing my girlfriend who made an end to our re...
1,disgust,[ No response.]
2,fear,Staying alone in a dark place.
3,shame,When I failed grade 7.
4,anger,I am a teacher in arts and crafts (boys from 1...


In [5]:
total_count = df['Emotions'].value_counts().sum()
print (total_count)

5366


In [6]:
df['Emotions'].value_counts()


Emotions
sadness    775
fear       770
guilt      767
shame      765
anger      764
joy        764
disgust    761
Name: count, dtype: int64

In [7]:
print (df['Emotions'])
print (df['Emotions'].shape)
print (df['Emotions'].values)

0       sadness
1       disgust
2          fear
3         shame
4         anger
         ...   
5361       fear
5362      anger
5363    sadness
5364    disgust
5365       fear
Name: Emotions, Length: 5366, dtype: object
(5366,)
['sadness' 'disgust' 'fear' ... 'sadness' 'disgust' 'fear']


In [8]:
emotion_labels = list (set (df['Emotions'].values))

In [9]:


def tokenize(text):
    translation_table = str.maketrans({c: f' {c} ' if not c.isalnum() else c for c in set(text)})  # creates translation
    # table(dictionary) with the built-in function maketrans, set(text) makes an unordered collection of unique
    # elements through set comprehension, a concise way to create sets
    tokenized_text = text.translate(translation_table)  # uses translation table to add whitespace around special
    # characters and punctuation
    return tokenized_text.strip().lower().split()  # split-tokens split on space, lower-tokens made all lowercase,
    # strip-and leading or trailing whitespaces are removed from string


#extract dict
text = ''.join(df['Text'].astype(str))
tokenized_text = tokenize(text) #all tokens, including repeating
#print (tokenized_text)
vocab = set(tokenized_text)
#print(vocab)

tokenized_text = []
for index, sentence in enumerate(df['Text']):
    emotion_label = df['Emotions'][index]
    sentence = tokenize(sentence)
    tokenized_text.append (sentence)
    #print(emotion_label, sentence)
#print (tokenized_text)


In [10]:
from math import log

collection = [token for instance in tokenized_text for token in instance]

#tf-idf

def calculate_tfidf(token, document):
    #tf = np.log(document.count(token) / len(document)) if token in document else 0 # 1+log(#oftimestokenindoc/total#termsindoc)
    tf = document.count(token) / len(document)
    idf = log (1 + (len(tokenized_text) / collection.count(token))) # log(#ofdocs/#oftimestokenappearsincollection)
    tfidf = tf * idf
    return tfidf

tfidf_scores_list = []
for sentence in tokenized_text:
        tfidf_scores = {token: calculate_tfidf(token, sentence) for token in set(sentence)}
        tfidf_scores_list.append (tfidf_scores)
#print (tfidf_scores_list)


In [11]:
from scipy.sparse import csr_matrix

# Construct a vocabulary list from the tfidf_scores_list
vocab = list(set(token for scores in tfidf_scores_list for token in scores.keys()))

# Construct a mapping from term to column index
term_to_col = {term: idx for idx, term in enumerate(vocab)}

# Convert the list of TF-IDF scores into a CSR sparse matrix
n_docs = len(tfidf_scores_list)
n_terms = len(vocab)
rows = []
cols = []
data = []
for doc_idx, scores in enumerate(tfidf_scores_list):
    for term, tfidf in scores.items():
        col_idx = term_to_col[term]
        rows.append(doc_idx)
        cols.append(col_idx)
        data.append(tfidf)

X_tfidf_sparse = csr_matrix((data, (rows, cols)), shape=(n_docs, n_terms))

print (X_tfidf_sparse)

  (0, 501)	0.2525889521228479
  (0, 534)	0.12137552202727434
  (0, 559)	0.11946792019026316
  (0, 610)	0.2202877841692488
  (0, 1061)	0.2525889521228479
  (0, 1432)	0.03134580378415336
  (0, 1828)	0.028851345353904934
  (0, 1972)	0.17721495308269722
  (0, 2014)	0.18800850810763658
  (0, 2023)	0.14504464952775478
  (0, 2102)	0.18800850810763658
  (0, 3129)	0.0963102394974555
  (0, 3559)	0.12600676567498867
  (0, 3594)	0.06415157025106248
  (0, 3683)	0.14288132351292446
  (0, 3863)	0.08191415255829469
  (0, 4056)	0.09740484719258474
  (0, 4163)	0.14091334153638377
  (0, 4170)	0.02827575741177872
  (0, 4374)	0.04381188587772471
  (0, 4438)	0.07520801777670834
  (0, 4658)	0.03721618287588566
  (0, 4804)	0.17112431203079617
  (0, 5472)	0.10904148218932437
  (0, 5717)	0.03333307662509752
  :	:
  (5365, 1119)	0.10884515860396833
  (5365, 1148)	0.024584628813797117
  (5365, 1364)	0.1972692802271394
  (5365, 1399)	0.0969728559032565
  (5365, 1432)	0.04844351493914611
  (5365, 1546)	0.0599260384

In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.sparse import csr_matrix

# Convert the list of tokenized sentences into a list of strings
"""corpus = [' '.join(sentence) for sentence in tokenized_text]

# Create a CountVectorizer to convert the corpus into a document-term matrix
count_vectorizer = CountVectorizer(vocabulary=vocab)
X_count = count_vectorizer.fit_transform(corpus)

# Create a TfidfTransformer to compute the TF-IDF scores
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_count)

# Convert the TF-IDF matrix to CSR format if needed
if not isinstance(X_tfidf, csr_matrix):
    X_tfidf = X_tfidf.tocsr()"""

#print (corpus)
#print(X_tfidf)

y = df['Emotions'].values


In [14]:
"""tfidf_df = pd.DataFrame(X_tfidf)

tfidf_df.to_csv('tfidf_representations.csv', index=False)

tfidf_df.to_excel('tfidf_representations.xlsx', index=False)"""

"tfidf_df = pd.DataFrame(X_tfidf)\n\ntfidf_df.to_csv('tfidf_representations.csv', index=False)\n\ntfidf_df.to_excel('tfidf_representations.xlsx', index=False)"

In [15]:
#%store X_tfidf
%store X_tfidf_sparse
%store y

UsageError: Unknown variable 'X_tfidf'
